Processamento de dados:
- criar tabela com todos os MQLs, com todas as informações disponíveis para cada um.
- criar uma coluna com quais MQLs fecharam acordo e quais foram perdidos.

In [29]:
import pandas as pd
import numpy as np

marketing_path = r'olist_marketing_qualified_leads_dataset.csv'
df_marketing = pd.read_csv(marketing_path)
print(df_marketing.shape)

closed_path = r'olist_closed_deals_dataset.csv'
df_closed= pd.read_csv(closed_path)
df_closed['contract'] = "accepted"
print(df_closed.shape)

(8000, 4)
(842, 15)


In [30]:
#join
df_total_mql = df_marketing.merge(df_closed, how='left', on='mql_id')

# replace NaN with 0
df_total_mql = df_total_mql.replace(np.nan, 0)

# fillna with 
df_total_mql['seller_id'].fillna(0, inplace=True)
df_total_mql.loc[(df_total_mql['seller_id'] == 0), 'contract'] = "lost"

print(df_total_mql.contract)

0           lost
1           lost
2           lost
3           lost
4       accepted
          ...   
7995        lost
7996        lost
7997        lost
7998        lost
7999    accepted
Name: contract, Length: 8000, dtype: object


In [31]:
#Qual foi a taxa de conversão total?
### Taxa de conversão total - somar todas a receita mensal e dividir pela quantidade total de mql
contract_series = df_total_mql['contract'].value_counts()
count_all = len(df_total_mql)
count_true = contract_series['accepted']

tx_conversao_total = (count_true / count_all) * 100

print("Qual foi a taxa de conversão total? ", tx_conversao_total, "%")


Qual foi a taxa de conversão total?  10.525 %


In [32]:
#Qual foi a taxa de conversão de cada origem?
df_origem_conv = df_total_mql.groupby(['origin', 'contract']).agg(count=('origin', 'count')).reset_index()
df_origem_accepted = df_origem_conv[df_origem_conv['contract'] == 'accepted'].rename(columns={'count': 'accepted'})
df_origem_lost = df_origem_conv[df_origem_conv['contract'] == 'lost'].rename(columns={'count': 'lost'})
df_origem_accepted.drop('contract', axis=1, inplace=True)
df_origem_lost.drop('contract', axis=1, inplace=True)

df_origem_conv_result = df_origem_accepted.merge(df_origem_lost, how='outer', on='origin')
df_origem_conv_result['taxa_conv'] = df_origem_conv_result['accepted'] / df_origem_conv_result['lost']

print(f'Taxa de conversão de cada origem: \n{df_origem_conv_result}') 

Taxa de conversão de cada origem: 
               origin  accepted  lost  taxa_conv
0                   0        14    46   0.304348
1      direct_traffic        56   443   0.126411
2             display         6   112   0.053571
3               email        15   478   0.031381
4      organic_search       271  2025   0.133827
5               other         4   146   0.027397
6   other_publicities         3    62   0.048387
7         paid_search       195  1391   0.140187
8            referral        24   260   0.092308
9              social        75  1275   0.058824
10            unknown       179   920   0.194565


In [33]:
#Qual foi a taxa de conversão de cada página inicial?
df_pagina_conv = df_total_mql.groupby(['landing_page_id', 'contract']).agg(count=('landing_page_id', 'count')).reset_index()
df_pagina_accepted = df_pagina_conv[df_pagina_conv['contract'] == 'accepted'].rename(columns={'count': 'accepted'})
df_pagina_lost = df_pagina_conv[df_pagina_conv['contract'] == 'lost'].rename(columns={'count': 'lost'})
df_pagina_accepted.drop('contract', axis=1, inplace=True)
df_pagina_lost.drop('contract', axis=1, inplace=True)

df_pagina_conv_result = df_pagina_accepted.merge(df_pagina_lost, how='outer', on='landing_page_id')

df_pagina_conv_result['taxa_conv'] = df_pagina_conv_result['accepted'] / df_pagina_conv_result['lost']
df_pagina_conv_result.fillna(0, inplace=True)

print(f'Taxa de conversão de cada pagina: \n{df_pagina_conv_result}') 

Taxa de conversão de cada pagina: 
                      landing_page_id  accepted  lost  taxa_conv
0    007f9098284a86ee80ddeb25d53e0af8       4.0  62.0   0.064516
1    0218f6be0b76aca72ab4d00ee9e8cf10       6.0  42.0   0.142857
2    04a75b75d5f4f3a16b0178fbe382da00       1.0   0.0   0.000000
3    0625ca637bf063abf254a66fe91fa84b       1.0   2.0   0.500000
4    090ffa1062e6dde353c36524563acefc       1.0   2.0   0.500000
..                                ...       ...   ...        ...
490  fc83c311415c9f67b6c7c87cf488b9c2       0.0   1.0   0.000000
491  fd660f5d9a62d444c76191307b3a0514       0.0  11.0   0.000000
492  fd7c4cdfbfb7964b441618d54f88c38b       0.0   1.0   0.000000
493  fd7d977e75d1ac5cfe6b93ffabb150b1       0.0   1.0   0.000000
494  ff3416719616ad4921acb8c74e21d6f0       0.0  11.0   0.000000

[495 rows x 4 columns]


In [34]:
# Para cada SR, qual a receita média declarada dos leads?
df_sr_receita = df_total_mql.groupby(['sr_id']).agg(receita=('declared_monthly_revenue', 'mean')).reset_index()
print('A receita média mensal declarada de cada Lead é: \n', df_sr_receita)

A receita média mensal declarada de cada Lead é: 
                                sr_id        receita
0                                  0       0.000000
1   060c0a26f19f4d66b42e0d8796688490       0.000000
2   068066e24f0c643eb1d089c7dd20cd73    7037.037037
3   0a0fb2b07d841f84fb6714e35c723075    6000.000000
4   2695de1affa7750089c0455f8ce27021    1694.915254
5   34d40cdaf94010a1d05b0d6212f9e909   19000.000000
6   495d4e95a8cf8bbf8b432b612a2aa328    6666.666667
7   4b339f9567d060bcea4f5136b9f5949e   27777.777778
8   4ef15afb4b2723d8f3d81e51ec7afefe  377293.233083
9   56bf83c4bb35763a51c2baab501b4c67   12500.000000
10  6565aa9ce3178a5caf6171827af3a9ba       0.000000
11  6aa3b86a83d784b05f0e37e26b20860d    8000.000000
12  85fc447d336637ba1df43e793199fbc8    1562.500000
13  9749123c950bf8363ace42cb1c2d0815  103571.428571
14  9ae085775a198122c5586fa830ff7f2b       0.000000
15  9d12ef1a7eca3ec58c545c678af7869c   43333.333333
16  9e4d1098a3b0f5da39b0bc48f9876645       0.000000
17  a8387c01a

In [35]:
# Para cada SR, quantos de cada business_type a pessoa converteu?
sr_business_conv = df_total_mql.query('sr_id != 0 and business_type != 0')
sr_business_conv = sr_business_conv.groupby(['sr_id', 'business_type', 'contract']).agg(count=('business_type', 'count')).reset_index()
sr_business_conv.drop('contract', axis=1, inplace=True)
print(f'Para cada SR, a pessoa converteu o seguinte número de business_type: \n {sr_business_conv}')

Para cada SR, a pessoa converteu o seguinte número de business_type: 
                                sr_id business_type  count
0   060c0a26f19f4d66b42e0d8796688490  manufacturer      9
1   060c0a26f19f4d66b42e0d8796688490      reseller     23
2   068066e24f0c643eb1d089c7dd20cd73  manufacturer      6
3   068066e24f0c643eb1d089c7dd20cd73      reseller     21
4   0a0fb2b07d841f84fb6714e35c723075         other      1
5   2695de1affa7750089c0455f8ce27021  manufacturer     16
6   2695de1affa7750089c0455f8ce27021      reseller     42
7   34d40cdaf94010a1d05b0d6212f9e909  manufacturer      1
8   34d40cdaf94010a1d05b0d6212f9e909         other      1
9   34d40cdaf94010a1d05b0d6212f9e909      reseller      8
10  495d4e95a8cf8bbf8b432b612a2aa328  manufacturer     14
11  495d4e95a8cf8bbf8b432b612a2aa328      reseller     47
12  4b339f9567d060bcea4f5136b9f5949e  manufacturer      2
13  4b339f9567d060bcea4f5136b9f5949e      reseller      6
14  4ef15afb4b2723d8f3d81e51ec7afefe  manufacturer     43
1

In [36]:
# Para cada SR, quantos de cada lead_behaviour_profile a pessoa converteu?
sr_lead_behaviour_conv = df_total_mql.query('sr_id != 0 and lead_behaviour_profile != 0')
sr_lead_behaviour_conv = sr_lead_behaviour_conv.groupby(['sr_id', 'lead_behaviour_profile', 'contract']).agg(count=('business_type', 'count')).reset_index()
sr_lead_behaviour_conv.drop('contract', axis=1, inplace=True)
print(f'Para cada SR, a pessoa converteu o seguinte número de lead_behaviour_profile: \n {sr_lead_behaviour_conv}')

Para cada SR, a pessoa converteu o seguinte número de lead_behaviour_profile: 
                                sr_id lead_behaviour_profile  count
0   060c0a26f19f4d66b42e0d8796688490                    cat     15
1   060c0a26f19f4d66b42e0d8796688490                  eagle      2
2   060c0a26f19f4d66b42e0d8796688490                  shark      1
3   060c0a26f19f4d66b42e0d8796688490                   wolf      6
4   068066e24f0c643eb1d089c7dd20cd73                    cat     11
..                               ...                    ...    ...
75  de63de0d10a6012430098db33c679b0b                   wolf      5
76  fbf4aef3f6915dc0c3c97d6812522f6a                    cat     30
77  fbf4aef3f6915dc0c3c97d6812522f6a                  eagle      9
78  fbf4aef3f6915dc0c3c97d6812522f6a                  shark      2
79  fbf4aef3f6915dc0c3c97d6812522f6a                   wolf      2

[80 rows x 3 columns]


In [37]:
# Para cada SR, quantos de cada lead_type a pessoa converteu?
sr_lead_type_conv = df_total_mql.query('sr_id != 0 and lead_type != 0')
sr_lead_type_conv = sr_lead_type_conv.groupby(['sr_id', 'lead_type', 'contract']).agg(count=('business_type', 'count')).reset_index()
sr_lead_type_conv.drop('contract', axis=1, inplace=True)
print(f'Para cada SR, a pessoa converteu o seguinte número de lead_type: \n {sr_lead_type_conv}')

Para cada SR, a pessoa converteu o seguinte número de lead_type: 
                                 sr_id        lead_type  count
0    060c0a26f19f4d66b42e0d8796688490         industry      8
1    060c0a26f19f4d66b42e0d8796688490          offline      1
2    060c0a26f19f4d66b42e0d8796688490  online_beginner      3
3    060c0a26f19f4d66b42e0d8796688490       online_big      2
4    060c0a26f19f4d66b42e0d8796688490    online_medium     15
..                                ...              ...    ...
113  fbf4aef3f6915dc0c3c97d6812522f6a  online_beginner      1
114  fbf4aef3f6915dc0c3c97d6812522f6a       online_big      8
115  fbf4aef3f6915dc0c3c97d6812522f6a    online_medium     30
116  fbf4aef3f6915dc0c3c97d6812522f6a     online_small      6
117  fbf4aef3f6915dc0c3c97d6812522f6a       online_top      2

[118 rows x 3 columns]


Perguntas a serem respondidas:
- Qual foi a taxa de conversão total?
- Qual foi a taxa de conversão de cada origem?
- Qual foi a taxa de conversão de cada página inicial?
- Para cada SR, qual a receita média declarada dos leads?
- Para cada SR, quantos de cada business_type a pessoa converteu?
- Para cada SR, quantos de cada lead_behaviour_profile a pessoa converteu?
- Para cada SR, quantos de cada lead_type a pessoa converteu?

In [39]:

print("Qual foi a taxa de conversão total? ", tx_conversao_total, "%")
print(f'\nTaxa de conversão de cada origem: \n{df_origem_conv_result}') 
print(f'\nTaxa de conversão de cada pagina: \n{df_pagina_conv_result}') 
print('\nA receita média mensal declarada de cada Lead é: \n', df_sr_receita)
print(f'\nPara cada SR, a pessoa converteu o seguinte número de business_type: \n {sr_business_conv}')
print(f'\nPara cada SR, a pessoa converteu o seguinte número de lead_behaviour_profile: \n {sr_lead_behaviour_conv}')
print(f'\nPara cada SR, a pessoa converteu o seguinte número de lead_type: \n {sr_lead_type_conv}')

Qual foi a taxa de conversão total?  10.525 %

Taxa de conversão de cada origem: 
               origin  accepted  lost  taxa_conv
0                   0        14    46   0.304348
1      direct_traffic        56   443   0.126411
2             display         6   112   0.053571
3               email        15   478   0.031381
4      organic_search       271  2025   0.133827
5               other         4   146   0.027397
6   other_publicities         3    62   0.048387
7         paid_search       195  1391   0.140187
8            referral        24   260   0.092308
9              social        75  1275   0.058824
10            unknown       179   920   0.194565

Taxa de conversão de cada pagina: 
                      landing_page_id  accepted  lost  taxa_conv
0    007f9098284a86ee80ddeb25d53e0af8       4.0  62.0   0.064516
1    0218f6be0b76aca72ab4d00ee9e8cf10       6.0  42.0   0.142857
2    04a75b75d5f4f3a16b0178fbe382da00       1.0   0.0   0.000000
3    0625ca637bf063abf254a66fe91fa

Extra:

Caso consiga resolver as questões acima com facilidade, aproveite a base "irmã" dessa para fazer outras explorações que você ache interessante!

https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

Lembre de separar novamente em 2 etapas: 
1. Processamento dos dados, para criar uma tabela "analítica".
2. Análise exploratória dos dados.